In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:


CY_file = "CY2024.csv"
df_cy = pd.read_csv(CY_file)


# Datetime of CY
df_cy['Cert Date'] = pd.to_datetime(df_cy['Cert Date'], format='%Y%m%d')
filtered_date = df_cy[(df_cy['Cert Date'] >= '2024-08-25') & (df_cy['Cert Date'] <= '2024-10-31')]
filtered_data = filtered_date[filtered_date['Type Carrier'] == 1]

selected_columns = [
    'Thursday', 'Serial No.', 'Type Shipm', 'Type Serv', 'Cert Date',
    'Type Carrier', 'Carrier Name', 'Grade', 'Grain', 'Destination', 'Subl/Carrs',
    'Field Office', 'Port', 'AMS Reg', 'FGIS Reg', 'Metric Ton'
]
filtered_data=filtered_data[selected_columns]
filtered_data = filtered_data.rename(columns={'Carrier Name': 'Vessel', 'Grain': 'Product'})
output_path = 'Filtered_CY2024.csv'
filtered_data.to_csv(output_path, index=False)
cy = filtered_data
print(f"Save it to: {output_path}")



kpler_file = "XXX_Extract_DataOpsChallenge_US_GrainsOilseeds_2024.xlsx"
df_kpler = pd.read_excel(kpler_file)

# Capitalize Product in Kpler data
df_kpler['Product'] = df_kpler['Product'].str.upper()

# SOYBEAN to SOYBEANS
df_kpler['Product'] = df_kpler['Product'].replace('SOYBEAN', 'SOYBEANS')

df_kpler['Date (origin)'] = pd.to_datetime(df_kpler['Date (origin)'], errors='coerce')
df_kpler['Date (destination)'] = pd.to_datetime(df_kpler['Date (destination)'], errors='coerce')
filtered_data = df_kpler[
    ((df_kpler['Date (origin)'] >= '2024-09-01') & (df_kpler['Date (origin)'] <= '2024-10-31'))
    # |
    # ((df_kpler['Date (destination)'] >= '2024-09-01') & (df_kpler['Date (destination)'] <= '2024-10-31'))
]
filtered_data = filtered_data.sort_values(by='Date (origin)')
filtered_data['Vessel'] = filtered_data['Vessel'].str.upper()
output_path = 'Filtered_Kpler_Sep_Oct.csv'
filtered_data.to_csv(output_path, index=False)
kpler = filtered_data



<ipython-input-2-ab5cdb4da3e1>:2: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cy = pd.read_csv(CY_file)


Save it to: Filtered_CY2024.csv


In [ ]:
def summarize_missing_values(dataframe, dataset_name):
    missing_summary = dataframe.isnull().sum().reset_index()
    missing_summary.columns = ['Column', 'Missing Values']
    missing_summary['Total Values'] = len(dataframe)
    missing_summary['Missing Percentage'] = (missing_summary['Missing Values'] / missing_summary['Total Values']) * 100
    missing_summary.insert(0, 'Dataset', dataset_name)
    return missing_summary



cy_missing_summary = summarize_missing_values(cy, 'CY')

kpler_missing_summary = summarize_missing_values(kpler, 'Kpler')

In [ ]:
cy_missing_summary

,Dataset,Column,Missing Values,Total Values,Missing Percentage
0,CY,Thursday,0,664,0.0
1,CY,Serial No.,0,664,0.0
2,CY,Type Shipm,0,664,0.0
3,CY,Type Serv,0,664,0.0
4,CY,Cert Date,0,664,0.0
5,CY,Type Carrier,0,664,0.0
6,CY,Vessel,0,664,0.0
7,CY,Grade,0,664,0.0
8,CY,Product,0,664,0.0
9,CY,Destination,0,664,0.0


In [ ]:
kpler_missing_summary

,Dataset,Column,Missing Values,Total Values,Missing Percentage
0,Kpler,Vessel,0,1008,0.000000
1,Kpler,Date (origin),0,1008,0.000000
2,Kpler,Origin,0,1008,0.000000
3,Kpler,Trade status,0,1008,0.000000
4,Kpler,Date (destination),36,1008,3.571429
5,Kpler,Destination,48,1008,4.761905
6,Kpler,Group,0,1008,0.000000
7,Kpler,Product,99,1008,9.821429
8,Kpler,Cargo (tons),0,1008,0.000000


In [ ]:
# Product Missing Data Processing
product_missing_data = kpler[kpler['Product'].isnull()]

# Based on Group distribution to fill in NaNs in the following code
group_distribution = product_missing_data['Group'].value_counts()


print("Product Missing rows：")
print(product_missing_data)

print("\nGroup Distribution：")
print(group_distribution)

Product Missing rows：
                Vessel       Date (origin)                        Origin  \
1145  ERNST OLDENDORFF 2024-09-01 06:30:00  Long Beach Dry Bulk Terminal   
1144  ERNST OLDENDORFF 2024-09-01 06:30:00  Long Beach Dry Bulk Terminal   
1132         PAC AGENA 2024-09-02 00:00:00                  EGT Longview   
1131         PAC AGENA 2024-09-02 00:00:00                  EGT Longview   
1122        SEA FALCON 2024-09-02 10:45:00                        Chipco   
...                ...                 ...                           ...   
160         OCEAN LION 2024-10-29 19:15:00                   Terre Haute   
155     GALAXY EXPRESS 2024-10-29 21:55:00                  EGT Longview   
154     GALAXY EXPRESS 2024-10-29 21:55:00                  EGT Longview   
153      LIBERTY EAGLE 2024-10-30 00:15:00                Wood Resources   
141        NORDIC OSLO 2024-10-30 21:06:00                KM Tampa Bulks   

     Trade status  Date (destination)               Destination  

In [ ]:
#Function for filling product only
def fill_missing_product(row):
    if pd.isnull(row['Product']):
        if row['Group'] == 'Grains/Oilseeds':
            return 'Unclear Grains/Oilseeds'
        elif row['Group'] == 'Grains':
            return 'Unclear Grains'
        elif row['Group'] == 'Oilseeds':
            return 'Unclear Oilseeds'
        else:
            return 'Unclear'  # default as Unclear
    return row['Product']

kpler['Product'] = kpler.apply(fill_missing_product, axis=1)

kpler_missing_summary = summarize_missing_values(kpler, 'Kpler')
kpler_missing_summary

,Dataset,Column,Missing Values,Total Values,Missing Percentage
0,Kpler,Vessel,0,1008,0.000000
1,Kpler,Date (origin),0,1008,0.000000
2,Kpler,Origin,0,1008,0.000000
3,Kpler,Trade status,0,1008,0.000000
4,Kpler,Date (destination),36,1008,3.571429
5,Kpler,Destination,48,1008,4.761905
6,Kpler,Group,0,1008,0.000000
7,Kpler,Product,0,1008,0.000000
8,Kpler,Cargo (tons),0,1008,0.000000


In [ ]:
# Fill missing 'Date (destination)' and 'Destination' with 'unknown'
kpler['Date (destination)'] = kpler['Date (destination)'].fillna('unknown')
kpler['Destination'] = kpler['Destination'].fillna('unknown')

In [ ]:
kpler_missing_summary = summarize_missing_values(kpler, 'Kpler')
kpler_missing_summary

,Dataset,Column,Missing Values,Total Values,Missing Percentage
0,Kpler,Vessel,0,1008,0.0
1,Kpler,Date (origin),0,1008,0.0
2,Kpler,Origin,0,1008,0.0
3,Kpler,Trade status,0,1008,0.0
4,Kpler,Date (destination),0,1008,0.0
5,Kpler,Destination,0,1008,0.0
6,Kpler,Group,0,1008,0.0
7,Kpler,Product,0,1008,0.0
8,Kpler,Cargo (tons),0,1008,0.0


In [ ]:
output_path = 'Filtered_Kpler_Sep_Oct.csv'
kpler.to_csv(output_path, index=False)

In [ ]:
# find two common set of vessels
unique_cy_vessels = set(cy['Vessel'])
unique_kpler_vessels = set(kpler['Vessel'])
common_vessels = unique_cy_vessels.intersection(unique_kpler_vessels)
print(f"Number of common vessels: {len(common_vessels)}")
print(f"Common vessels are: {common_vessels}")



Number of common vessels: 397
Common vessels are: {'COLUMBIA RIVER', 'MARSANNE', 'CARLA C', 'UBC SANTOS', 'VITALITY DIVA', 'EVA PARIS', 'PERSEUS', 'SPAR SCORPIO', 'ULTRA VISION', 'V UNO', 'VINDONISSA', 'MADRID', 'AIANTAS', 'REMY ENTERPRISE', 'NORDORINOCO', 'TRES FELICES', 'GLOBAL GREEN', 'BELMONDO', 'THEMISTOCLES', 'WHEAT WEIFANG', 'SAMATAN', 'NAVISION VINCENTZ', 'IOANNIS K', 'SSI DOMINION', 'BALSA 86', 'CAPTAIN D', 'GUO YUAN 8', 'SILVERFJORD', 'IONIC SMYRNI', 'SARAH H', 'LYNX', 'ESTELA CLAIRE', 'NORD CHESAPEAKE', 'ES KURE', 'PAC AGENA', 'REGGEBORG', 'DARYA AUM', 'GUO YUAN 10', 'ALMA', 'PRINCESS A', 'ORIENT TIDE', 'CSSC TAI YUAN', 'PACIFIC PIONEER', 'MEDITERRANEAN SPIRIT', 'SHANDONG FU YI', 'GLOBAL HOPE', 'BALSA 92', 'AL SAAD', 'KRINI', 'INFINITY SKY', 'EAGLE TRADER', 'TASSOS N', 'MAROUDIO', 'IONA ISLAND', 'ATHINA L', 'SUNRISE JADE', 'AFRICAN JOSEPH R', 'ISABELA ISLAND', "FIRST D'YQUEM", 'MALENA', 'OCEAN AMBITION', 'DARYA RASHMI', 'INDIGO LUFFY', 'OCEAN JADE', 'JULIETTE', 'AQUAVITA TRA

In [ ]:
print("Numbers of Unique CY Vessels: ", len(unique_cy_vessels))
print("Numbers of Unique Kpler Vessels: ", len(unique_kpler_vessels))

Numbers of Unique CY Vessels:  451
Numbers of Unique Kpler Vessels:  475


In [ ]:
# Find vessels in kpler but not in common
kpler_only_vessels = unique_kpler_vessels - common_vessels
print("\nVessels in Kpler but not in common:")
kpler_only_vessels_df = pd.DataFrame({'Kpler Only Vessels': list(kpler_only_vessels)})
print(kpler_only_vessels_df)

# Find vessels in cy but not in common
cy_only_vessels = unique_cy_vessels - common_vessels
print("\nVessels in CY but not in common:")
cy_only_vessels_df = pd.DataFrame({'CY Only Vessels': list(cy_only_vessels)})
print(cy_only_vessels_df)


Vessels in Kpler but not in common:
         Kpler Only Vessels
0               OSLO BULK 9
1                KANG HUANG
2      INDUSTRIALCHALLENGER
3                BUNUN HERO
4                OCEAN LION
..                      ...
73                  JANET C
74                   ILIANA
75              VEGA DABLAM
76                TAC IMOLA
77  RT. HON. PAUL J. MARTIN

[78 rows x 1 columns]

Vessels in CY but not in common:
        CY Only Vessels
0             CL DALIAN
1            ISTANBUL-M
2            ETG UBUNTU
3           TAI STAMINA
4        AMBER ETERNITY
5             REESTBORG
6                BOGDAN
7      Federal Kivalina
8       HUA  SHENG  HAI
9           GLOBAL SAGE
10                UNITY
11    CLIPPER BARI-STAR
12           FRASERBORG
13             SERENITY
14         ARUNA CENGIZ
15        DORIC TRIDENT
16        FEDERAL CLYDE
17  STRATEGIC FORTITUDE
18    VAN LEOPARD-LOT 2
19          SN SERENITY
20            MONT FORT
21                Tasos
22          NORD A

In [ ]:
# Filtered Data with common vessels names
common_vessels=list(common_vessels)
cy_common_data = cy[cy['Vessel'].isin(common_vessels)]
kpler_common_data = kpler[kpler['Vessel'].isin(common_vessels)]


In [ ]:
#Matching method 2
matched_data = pd.merge(
    cy_common_data[['Vessel', 'Cert Date', 'Product', 'Metric Ton']],
    kpler_common_data[['Vessel', 'Date (origin)', 'Product', 'Cargo (tons)']],
    left_on=['Vessel', 'Product'],
    right_on=['Vessel', 'Product'],
    how='outer',
    suffixes=('_CY', '_Kpler')
)

In [ ]:
matched_data

,Vessel,Cert Date,Product,Metric Ton,Date (origin),Cargo (tons)
0,THEMISTOCLES,2024-08-25,WHEAT,11891.0,2024-09-01 15:00:00,13444.0
1,THEMISTOCLES,2024-08-25,WHEAT,6457.0,2024-09-01 15:00:00,13444.0
2,THEMISTOCLES,2024-09-01,WHEAT,32760.0,2024-09-01 15:00:00,13444.0
3,SHANDONG HAI XING,2024-08-25,CORN,62742.0,NaT,NaN
4,KYRA ZAFIRA,2024-08-26,CORN,72599.0,NaT,NaN
...,...,...,...,...,...,...
1204,TURBO S,NaT,Unclear Grains/Oilseeds,NaN,2024-10-27 03:26:00,36373.0
1205,RUI NING 22,NaT,Unclear Grains/Oilseeds,NaN,2024-10-28 15:49:00,33993.0
1206,GALAXY EXPRESS,NaT,Unclear Oilseeds,NaN,2024-10-29 21:55:00,20085.0
1207,GALAXY EXPRESS,NaT,Unclear Grains,NaN,2024-10-29 21:55:00,20086.0


In [ ]:

def resolve_unclear_in_kpler(cy_data, kpler_common_data, matched_data):

    """
    This function resolves unclear product information in the kpler_common_data dataset
    by inferring data from the cy_data dataset based on the associated vessels and
    grain types.

    - A copy of kpler_common_data is created for manipulation.
    - Each vessel in cy_data is mapped to the unique list of Grain types associated
      with it.
    - Rows in the kpler_common_data dataset with unclear product information are updated
      based on the Grain types associated with the vessel:
        - If the vessel has a unique Grain type in cy_data, the product is updated to
          that Grain type, marked as "(unclear)".
        - If the vessel has multiple Grain types in cy_data, the product is marked as
          "Unclear+Original" to indicate ambiguity.
    """

    # Copy kpler_common_data for creating kpler_common_data_1
    kpler_common_data_1 = kpler_common_data.copy()

    # Map each vessel in CY data to its associated unique Grain types
    vessel_grain_map = (
        cy_data.groupby('Vessel')['Product']
        .apply(lambda grains: list(grains.dropna().unique()))
        .to_dict()
    )

    # Iterate over each row in kpler_common_data_1
    for index, row in kpler_common_data_1.iterrows():
        vessel = row['Vessel']
        product = row['Product']

        # If the product starts with "Unclear" or is NaN
        if pd.isna(product) or 'Unclear' in str(product):
            if vessel in vessel_grain_map:
                grains = vessel_grain_map[vessel]

                # Check if the vessel in CY data has a unique Grain type
                if len(grains) == 1:
                    # If unique, infer the product as the single Grain type
                    kpler_common_data_1.at[index, 'Product'] = f"{grains[0]} (unclear)"
                else:
                    # If not unique, mark as Unclear+Original
                    kpler_common_data_1.at[index, 'Product'] = f"{product}"

    return kpler_common_data_1

# 调用函数修正数据
kpler_common_data_1 = resolve_unclear_in_kpler(cy_common_data, kpler_common_data, matched_data)



In [ ]:
kpler_common_data_1['Product'].unique()

array(['WHEAT (unclear)', 'WHEAT', 'SOYBEANS', 'SOYBEAN MEAL',
       'DRIED DISTILLER GRAINS', 'CORN', 'SORGHUM', 'CORN (unclear)',
       'Unclear Grains/Oilseeds', 'SOYBEANS (unclear)'], dtype=object)

In [ ]:
# kpler common 1 Product, add one new unified product column
# CORN (unclear) => CORN，XX（unclear）=>XX

def unify_product_column(df):
    """
    Unifies the 'Product' column in the DataFrame based on specified rules.
    """
    df['Unified Product'] = df['Product'].copy()  # Initialize with original values

    # Apply transformation rules
    df['Unified Product'] = df['Unified Product'].str.replace(r'(.*) \(unclear\)', r'\1', regex=True)

    return df

# Apply the function to your DataFrame
kpler_common_data_1 = unify_product_column(kpler_common_data_1)
kpler_common_data_1

,Vessel,Date (origin),Origin,Trade status,Date (destination),Destination,Group,Product,Cargo (tons),Unified Product
1145,ERNST OLDENDORFF,2024-09-01 06:30:00,Long Beach Dry Bulk Terminal,Delivered,2024-09-29 23:04:00,Hanshin Silo (Kobe),Grains/Oilseeds,WHEAT (unclear),6721,WHEAT
1144,ERNST OLDENDORFF,2024-09-01 06:30:00,Long Beach Dry Bulk Terminal,Delivered,2024-10-01 22:25:00,Nippon Flour (Kobe),Grains/Oilseeds,WHEAT (unclear),8160,WHEAT
1142,MADRID,2024-09-01 07:05:00,Ama,Delivered,2024-09-06 12:45:00,Shell Kingston Refinery,Grains,WHEAT,10014,WHEAT
1143,MADRID,2024-09-01 07:05:00,Ama,Delivered,2024-09-06 12:45:00,Shell Kingston Refinery,Grains,WHEAT,10013,WHEAT
1141,ASTRA CENTAURUS,2024-09-01 08:00:00,Baton Rouge,Delivered,2024-10-04 22:20:00,Oran,Oilseeds,SOYBEANS,52666,SOYBEANS
...,...,...,...,...,...,...,...,...,...,...
143,UBC SALERNO,2024-10-30 16:23:00,Ama,Delivered,2024-11-02 16:05:00,Tuxpan Grains,Grains,WHEAT,15760,WHEAT
142,UBC SALERNO,2024-10-30 16:23:00,Ama,Delivered,2024-11-02 16:05:00,Tuxpan Grains,Grains,CORN,15759,CORN
140,TOLMI,2024-10-30 21:35:00,United Grain Portland,In Transit,2024-11-27 15:00:00,Guangzhou,Oilseeds,SOYBEANS,36817,SOYBEANS
139,ES KURE,2024-10-30 22:23:00,St Charles,In Transit,2024-11-10 01:44:00,Acajutla,Grains,WHEAT,15964,WHEAT


In [ ]:
kpler_common_data_1['Unified Product'].unique()

array(['WHEAT', 'SOYBEANS', 'SOYBEAN MEAL', 'DRIED DISTILLER GRAINS',
       'CORN', 'SORGHUM', 'Unclear Grains/Oilseeds'], dtype=object)

In [ ]:
cy_agg = cy_common_data.groupby(['Vessel', 'Product'])['Metric Ton'].sum().reset_index()
cy_agg

,Vessel,Product,Metric Ton
0,ADELINA,WHEAT,29700
1,AEOLIAN BREEZE,SOYBEANS,57966
2,AEOLIAN VISION,SOYBEANS,68599
3,AFRICAN BATELEUR,SOYBEANS,60497
4,AFRICAN JACANA,CORN,68198
...,...,...,...
469,YAESU,CORN,61842
470,YANNIS PITTAS,WHEAT,12963
471,YASA KYOTO,CORN,16034
472,YASA MARS,SORGHUM,58571


In [ ]:
# Use new filtered kpler data with predict Product type and analyse the tons numbers again:
# 1. Numerical Gap: Based on Grain, Vessel aggregate CY and Kpler tons

kpler_agg_1 = (
    kpler_common_data_1.groupby(['Vessel', 'Unified Product'])['Cargo (tons)']
    .sum()
    .reset_index()
    .rename(columns={'Unified Product': 'Product', 'Cargo (tons)': 'Kpler Cargo Tons'})
)
kpler_agg_1

,Vessel,Product,Kpler Cargo Tons
0,ADELINA,WHEAT,32801
1,AEOLIAN BREEZE,SOYBEANS,70283
2,AEOLIAN VISION,SOYBEANS,72585
3,AFRICAN BATELEUR,SOYBEANS,59979
4,AFRICAN JACANA,CORN,79316
...,...,...,...
496,YAESU,CORN,58983
497,YANNIS PITTAS,WHEAT,23349
498,YASA KYOTO,CORN,36043
499,YASA MARS,SORGHUM,54973


In [ ]:
# Aggregate CY and Kpler
quantity_comparison_1 = pd.merge(
    cy_agg,
    kpler_agg_1,
    on=['Vessel', 'Product'],
    how='outer'
)
quantity_comparison_1

,Vessel,Product,Metric Ton,Kpler Cargo Tons
0,ADELINA,WHEAT,29700.0,32801.0
1,AEOLIAN BREEZE,SOYBEANS,57966.0,70283.0
2,AEOLIAN VISION,SOYBEANS,68599.0,72585.0
3,AFRICAN BATELEUR,SOYBEANS,60497.0,59979.0
4,AFRICAN JACANA,CORN,68198.0,79316.0
...,...,...,...,...
510,UNITY ODYSSEY,SORGHUM,NaN,18037.0
511,WARMIA,DRIED DISTILLER GRAINS,NaN,4355.0
512,WARMIA,Unclear Grains/Oilseeds,NaN,37571.0
513,WESTERN FUJI,DRIED DISTILLER GRAINS,NaN,19050.0


In [ ]:
# quantity_comparison_1['Tons Difference'] = quantity_comparison_1['CY Metric Ton'] - quantity_comparison_1['Kpler Cargo Tons']
#  if CY metric tons exist or just directly note it as +10^7

quantity_comparison_1['Tons Difference'] = quantity_comparison_1['Metric Ton'] - quantity_comparison_1['Kpler Cargo Tons']

quantity_comparison_1['Tons Difference Ratio'] = np.where(
    quantity_comparison_1['Metric Ton'].notna(),
    quantity_comparison_1['Tons Difference'].abs() / quantity_comparison_1['Metric Ton'],
    10**7
)

quantity_comparison_1['Anomalous Tons'] = quantity_comparison_1['Tons Difference Ratio'] > 0.15
quantity_comparison_1

,Vessel,Product,Metric Ton,Kpler Cargo Tons,Tons Difference,Tons Difference Ratio,Anomalous Tons
0,ADELINA,WHEAT,29700.0,32801.0,-3101.0,1.044108e-01,False
1,AEOLIAN BREEZE,SOYBEANS,57966.0,70283.0,-12317.0,2.124866e-01,True
2,AEOLIAN VISION,SOYBEANS,68599.0,72585.0,-3986.0,5.810580e-02,False
3,AFRICAN BATELEUR,SOYBEANS,60497.0,59979.0,518.0,8.562408e-03,False
4,AFRICAN JACANA,CORN,68198.0,79316.0,-11118.0,1.630253e-01,True
...,...,...,...,...,...,...,...
510,UNITY ODYSSEY,SORGHUM,NaN,18037.0,NaN,1.000000e+07,True
511,WARMIA,DRIED DISTILLER GRAINS,NaN,4355.0,NaN,1.000000e+07,True
512,WARMIA,Unclear Grains/Oilseeds,NaN,37571.0,NaN,1.000000e+07,True
513,WESTERN FUJI,DRIED DISTILLER GRAINS,NaN,19050.0,NaN,1.000000e+07,True


In [ ]:

# Calculate statistics on 'quantity_comparison_1'
total_vessels = len(quantity_comparison_1['Vessel'].unique())
anomalous_vessels = quantity_comparison_1[quantity_comparison_1['Anomalous Tons'] == True]['Vessel'].unique()


print(f"Total number of vessels: {total_vessels}")
print(f"Number of anomalous vessels: {len(anomalous_vessels)}")
print(f"Vessels with anomalous tons (True): {anomalous_vessels}")

#  Further statistics and analysis
# Count of anomalous and non-anomalous vessels
anomalous_count = quantity_comparison_1['Anomalous Tons'].sum()
non_anomalous_count = len(quantity_comparison_1) - anomalous_count
print(f"\nAnomalous vessels count: {anomalous_count}")
print(f"Non-anomalous vessels count: {non_anomalous_count}")

# Group by 'Anomalous Tons' and get counts
anomalous_counts = quantity_comparison_1.groupby('Anomalous Tons')['Vessel'].count()
print(f"\nAnomalous Tons Counts:\n{anomalous_counts}")


Total number of vessels: 397
Number of anomalous vessels: 160
Vessels with anomalous tons (True): ['AEOLIAN BREEZE' 'AFRICAN JACANA' 'AFRICAN TEAL' 'AMFITRITI'
 'AMIS LEADER' 'AQUAVITA TRADER' 'ARAMIS' 'ARIANDA' 'ASTRA CENTAURUS'
 'ATLANTIC TSUBAKI' 'ATTIKI SB' 'BALSA 87' 'BALSA 92' 'BALSA 95'
 'BBC SATURN' 'BEL AIR' 'BELLIGHT' 'BELMONDO' 'BLISS' 'BOLD GUARDIAN'
 'BONA' 'BRAVO' 'BUNUN TREASURE' 'CARLA C' 'CARMENCITA' 'CENTURION'
 'CENTURY ROYAL' 'CHARLIE' 'CHESTNUT' 'CITRUS TIGER' 'CMB JORDAENS'
 'CP CHONGQING' 'CSL ASSINIBOINE' 'DARYA AUM' 'DRAFTDODGER' 'EAGLE TRADER'
 'EMMA JANNEKE' 'ES KURE' 'ES LEADER' 'ETHRA 1' 'EVER RADIANCE'
 'FEDERAL BERING' 'FEDERAL PASSION' 'FERMITA' "FIRST D'YQUEM"
 'FORTUNE WING' 'FREE STATE' 'FRIEDERIKE' 'FRIEDRICH SCHULTE'
 'GALAXY EXPRESS' 'GDYNIA' 'GENCO PICARDY' 'GLOBAL GREEN' 'GREAT FAITH'
 'GREEN WINDS' 'GUO YUAN 12' 'HAKATA QUEEN' 'HOUTMANGRACHT' 'HSL VARNA'
 'INTERLINK SOLIDITY' 'JAMNO' 'JESSICA B' 'JULIETTE' 'JUNO' 'KOM' 'KOTOR'
 'LA BAMBA' 'LEM G

In [ ]:
def find_closest_match(row):

    relevant_cy_records = cy_common_data[
        (cy_common_data['Vessel'] == row['Vessel'])
    ]
    if not relevant_cy_records.empty:

        closest_row = relevant_cy_records.iloc[
            (relevant_cy_records['Cert Date'] - row['Date (origin)']).abs().argsort()[:1]
        ]
        return closest_row.iloc[0]['Cert Date'], closest_row.iloc[0].name
    return pd.NaT, None


# Apply the matching function
time_matched = pd.DataFrame()
time_matches = kpler_common_data_1.apply(find_closest_match, axis=1)

In [ ]:
kpler_common_data_1_test = kpler_common_data_1.copy()
kpler_common_data_1_test['Closest Cert Date'] = time_matches.apply(lambda x: x[0])  # Extract closest Cert Date
kpler_common_data_1_test['CY Row Index'] = time_matches.apply(lambda x: x[1])  # Extract matched row index in CY


In [ ]:
kpler_common_time = kpler_common_data_1_test

In [ ]:
kpler_common_time['Time Difference(Days)'] = (kpler_common_time['Closest Cert Date'] - kpler_common_time['Date (origin)']).dt.days
kpler_common_time['anomalous times'] = ((kpler_common_time['Time Difference(Days)'] > 2) | (kpler_common_time['Time Difference(Days)'] < -2))
kpler_common_time

,Vessel,Date (origin),Origin,Trade status,Date (destination),Destination,Group,Product,Cargo (tons),Unified Product,Closest Cert Date,CY Row Index,Time Difference(Days),anomalous times
1145,ERNST OLDENDORFF,2024-09-01 06:30:00,Long Beach Dry Bulk Terminal,Delivered,2024-09-29 23:04:00,Hanshin Silo (Kobe),Grains/Oilseeds,WHEAT (unclear),6721,WHEAT,2024-09-07,16878,5,True
1144,ERNST OLDENDORFF,2024-09-01 06:30:00,Long Beach Dry Bulk Terminal,Delivered,2024-10-01 22:25:00,Nippon Flour (Kobe),Grains/Oilseeds,WHEAT (unclear),8160,WHEAT,2024-09-07,16878,5,True
1142,MADRID,2024-09-01 07:05:00,Ama,Delivered,2024-09-06 12:45:00,Shell Kingston Refinery,Grains,WHEAT,10014,WHEAT,2024-08-31,16455,-2,False
1143,MADRID,2024-09-01 07:05:00,Ama,Delivered,2024-09-06 12:45:00,Shell Kingston Refinery,Grains,WHEAT,10013,WHEAT,2024-08-31,16455,-2,False
1141,ASTRA CENTAURUS,2024-09-01 08:00:00,Baton Rouge,Delivered,2024-10-04 22:20:00,Oran,Oilseeds,SOYBEANS,52666,SOYBEANS,2024-08-31,16457,-2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,UBC SALERNO,2024-10-30 16:23:00,Ama,Delivered,2024-11-02 16:05:00,Tuxpan Grains,Grains,WHEAT,15760,WHEAT,2024-10-30,21125,-1,False
142,UBC SALERNO,2024-10-30 16:23:00,Ama,Delivered,2024-11-02 16:05:00,Tuxpan Grains,Grains,CORN,15759,CORN,2024-10-30,21125,-1,False
140,TOLMI,2024-10-30 21:35:00,United Grain Portland,In Transit,2024-11-27 15:00:00,Guangzhou,Oilseeds,SOYBEANS,36817,SOYBEANS,2024-10-30,21151,-1,False
139,ES KURE,2024-10-30 22:23:00,St Charles,In Transit,2024-11-10 01:44:00,Acajutla,Grains,WHEAT,15964,WHEAT,2024-10-30,21200,-1,False


In [ ]:
# prompt: kpler_common_time 给我一个最后一列的异常值的计数，统计，true/false的百分比，然后打印出时间比对不上的vessels名称

# Calculate statistics on 'kpler_common_time'
total_vessels_time = len(kpler_common_time['Vessel'].unique())
anomalous_vessels_time = kpler_common_time[kpler_common_time['anomalous times'] == True]['Vessel'].unique()

print(f"Total number of vessels: {total_vessels_time}")
print(f"Number of vessels with anomalous times: {len(anomalous_vessels_time)}")

# Percentage of True/False in 'anomalous times' column
true_percentage = (kpler_common_time['anomalous times'].sum() / len(kpler_common_time)) * 100
false_percentage = 100 - true_percentage

print(f"\nPercentage of anomalous times (True): {true_percentage:.2f}%")
print(f"Percentage of non-anomalous times (False): {false_percentage:.2f}%")

# Print vessels with time mismatches
print("\nVessels with time mismatches:")
print(kpler_common_time[kpler_common_time['anomalous times'] == True]['Vessel'].unique())

Total number of vessels: 397
Number of vessels with anomalous times: 40

Percentage of anomalous times (True): 8.29%
Percentage of non-anomalous times (False): 91.71%

Vessels with time mismatches:
['ERNST OLDENDORFF' 'JULIETTE' 'GENCO PICARDY' 'BOLD GUARDIAN' 'BELMONDO'
 'OCEANLOVE' 'STAMFORD PIONEER' 'NIRIIS' 'SEAMASTER' 'VIRTUOUS STRIKER'
 'WARMIA' 'INTERLINK SOLIDITY' 'UBC TAMPA' 'FAIRFIELD EAGLE' 'AOM GAIA'
 'CARMENCITA' 'LIBERTY GLORY' 'BALTIC MANTIS' 'BUNUN TREASURE'
 'MARATHA PRESTIGE' 'VINDONISSA' 'OKINAWA' 'GENCO HUNTER' 'OCEAN JADE'
 'MADRID' 'ATHINA L' 'ARKLOW SPRAY' 'GLYFADA I' 'CAPRICORN CONFIDENCE'
 'JUNO' 'SELO' 'KT BIRDIE' 'INDIGO BREEZE' 'AMIS ACE' 'ROERBORG'
 'UBC SANTOS' 'KOTOR' 'ETHRA 1' 'SAN GEORGE' 'LIBERTY EAGLE']


Part2 after receiveing reviews


In [ ]:
cy_common_data


,Thursday,Serial No.,Type Shipm,Type Serv,Cert Date,Type Carrier,Vessel,Grade,Product,Destination,Subl/Carrs,Field Office,Port,AMS Reg,FGIS Reg,Metric Ton
15930,20240829,643599,BU,IW,2024-08-25,1,THEMISTOCLES,1,WHEAT,TAIWAN,13,PACIFICNW,COLUMBIA R.,PACIFIC,WEST COAST,11891
15931,20240829,643600,BU,IW,2024-08-25,1,THEMISTOCLES,1,WHEAT,TAIWAN,7,PACIFICNW,COLUMBIA R.,PACIFIC,WEST COAST,6457
15938,20240829,643609,BU,IW,2024-08-25,1,SHANDONG HAI XING,3 O/B,CORN,JAPAN,25,PACIFICNW,COLUMBIA R.,PACIFIC,WEST COAST,62742
16006,20240829,643704,BU,IW,2024-08-26,1,KYRA ZAFIRA,2 O/B,CORN,SPAIN,55,NEW ORLEANS,MISSISSIPPI R.,GULF,EAST GULF,72599
16033,20240829,643736,BU,IW,2024-08-27,1,MAPLE STAR,2 O/B,CORN,MEXICO,24,NEW ORLEANS,MISSISSIPPI R.,GULF,EAST GULF,31999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21201,20241031,649153,BU,IW,2024-10-30,1,ES KURE,2 O/B,CORN,EL SALVADOR,10,NEW ORLEANS,MISSISSIPPI R.,GULF,EAST GULF,12151
21203,20241031,649155,BU,IW,2024-10-29,1,DAIWAN ELEGANCE,2 O/B,SOYBEANS,PERU,12,NEW ORLEANS,MISSISSIPPI R.,GULF,EAST GULF,18699
21204,20241031,649156,BU,IW,2024-10-28,1,DAIWAN ELEGANCE,2 O/B,WHEAT,PERU,10,NEW ORLEANS,MISSISSIPPI R.,GULF,EAST GULF,15245
21531,20241031,649535,BU,IW,2024-10-30,1,RUI NING 22,2 O/B,SOYBEANS,CHINA,47,NEW ORLEANS,MISSISSIPPI R.,GULF,EAST GULF,64999


In [ ]:
kpler_common_data

,Vessel,Date (origin),Origin,Trade status,Date (destination),Destination,Group,Product,Cargo (tons)
1145,ERNST OLDENDORFF,2024-09-01 06:30:00,Long Beach Dry Bulk Terminal,Delivered,2024-09-29 23:04:00,Hanshin Silo (Kobe),Grains/Oilseeds,Unclear Grains/Oilseeds,6721
1144,ERNST OLDENDORFF,2024-09-01 06:30:00,Long Beach Dry Bulk Terminal,Delivered,2024-10-01 22:25:00,Nippon Flour (Kobe),Grains/Oilseeds,Unclear Grains/Oilseeds,8160
1142,MADRID,2024-09-01 07:05:00,Ama,Delivered,2024-09-06 12:45:00,Shell Kingston Refinery,Grains,WHEAT,10014
1143,MADRID,2024-09-01 07:05:00,Ama,Delivered,2024-09-06 12:45:00,Shell Kingston Refinery,Grains,WHEAT,10013
1141,ASTRA CENTAURUS,2024-09-01 08:00:00,Baton Rouge,Delivered,2024-10-04 22:20:00,Oran,Oilseeds,SOYBEANS,52666
...,...,...,...,...,...,...,...,...,...
143,UBC SALERNO,2024-10-30 16:23:00,Ama,Delivered,2024-11-02 16:05:00,Tuxpan Grains,Grains,WHEAT,15760
142,UBC SALERNO,2024-10-30 16:23:00,Ama,Delivered,2024-11-02 16:05:00,Tuxpan Grains,Grains,CORN,15759
140,TOLMI,2024-10-30 21:35:00,United Grain Portland,In Transit,2024-11-27 15:00:00,Guangzhou,Oilseeds,SOYBEANS,36817
139,ES KURE,2024-10-30 22:23:00,St Charles,In Transit,2024-11-10 01:44:00,Acajutla,Grains,WHEAT,15964


In [ ]:
# Only Keep Wheat, Corn, Soybeans


In [ ]:
# kpler api
!pip install kpler.sdk